# 📌 How to use this notebook (please read once)

Before you start any exercise:

1. Go to the top menu and click Run → Run all

![colab_top_bar_run](https://raw.githubusercontent.com/Haross/DB_pics_nt/main/colab_top_bar_run_all.png)

2. Wait about **1 minute** while everything loads

3. Once everything finishes loading, scroll down to start working with the notebook.

> 👉 **You need to do this once per session (each time you open the notebook).**





## ✍️ Writing and running SQL queries

In this notebook you will see a SQL editor like the one below:

![SQL_editor_run_button](https://raw.githubusercontent.com/Haross/DB_pics_nt/main/colab_sql_gui.png)


Type your SQL queries inside the provided GUI

> Toolbar buttons (in the SQL editor)
>
> * ▶ Blue Run query: runs your SQL and shows results.
>
> * ⟲ Revert to last saved: restores the last saved query for this exercise.
>
> * 🧹 Clear results: clears the displayed output (does not delete data).
>
> * ⌫ Clear editor: clears the query text (does not delete data or history).


Do not edit regular code cells unless explicitly asked

### About the validator

If the validator says your query is incorrect but you believe it is correct:

* Don’t panic 🙂

* The validator is a guideline, not a judge

* In rare cases, a correct query may not be recognized


## 🧠 About the SQL environment setup (do not edit)

You will see a section called “SQL Environment Setup (do not edit)”.

You do not need to understand or modify this

This code is required for the notebook to work

It runs automatically when you click **Run all**

> 👉 **Ignore the section SQL environment setup completely.**

## 🔄 If the SQL editor does not let you type

Sometimes the SQL editor may not respond.

If that happens:

1. Click the ▶ Play button in that same cell that appears at the top-left of the cell. As in the below picture

![SQL_editor_run_button](https://raw.githubusercontent.com/Haross/DB_pics_nt/main/colab_sql_run_Cell_button.png)

2. The editor will refresh and work normally




## 📂 Viewing your previous queries (optional)

After you submit your first query, new files will appear on the left sidebar:

* sql_query_log.csv → latest query per exercise

* sql_query_log_history.csv → all queries tried for each exercise

> ⚠️ These files do not appear until you run at least one query.

You do not need to open these files unless you are curious.

## 🚫 About hidden code

Some cells hide code and show a “Show code” button.

* Do not open these

* If you open one by accident:

  1. Click the cell

  2. Go to View → Show/Hide Code to hide it again

![SQL_editor_run_button](https://raw.githubusercontent.com/Haross/DB_pics_nt/main/colab_view_show_hide_cells.png)


# **SQL Environment Setup (do not edit)**

In [1]:
# @title
%%capture
!mkdir -p notebook_lib
!wget -q -O notebook_lib/sql_runner.py \
  https://raw.githubusercontent.com/Haross/sql_notebook/8021f5c05b7d973b8db549a1398a3c9a5c7829d5/notebook_lib/sql_runner.py
!wget -q -O notebook_lib/validators.py \
  https://raw.githubusercontent.com/Haross/sql_notebook/7baff2c6485cdf641cabcdb55d92a51317cd18b9/notebook_lib/validators.py

from notebook_lib.sql_runner import make_sql_runner
from notebook_lib.validators import make_df_validator_nospoilers, check_process_rules

import sqlite3
import pandas as pd
from pathlib import Path


In [2]:
# @title
%%capture
DB_FILE = 'class.db'

if DB_FILE != ":memory:":
    Path(DB_FILE).unlink(missing_ok=True)

conn = sqlite3.connect(DB_FILE)
conn.execute("PRAGMA foreign_keys = ON;")

conn.executescript(r'''
DROP TABLE IF EXISTS director;
DROP TABLE IF EXISTS movie;
-- Create table

CREATE TABLE director (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    birth_year INTEGER
);

CREATE TABLE movie (
    id INTEGER PRIMARY KEY,
    title TEXT NOT NULL,
    production_year INTEGER,
    director_id INTEGER,
    FOREIGN KEY (director_id) REFERENCES director(id)
);

-- Insert rows
INSERT INTO director (id, name, birth_year) VALUES
(1, 'Alfred Hitchcock', 1899),
(2, 'Steven Spielberg', 1946),
(3, 'Woody Allen', 1935),
(4, 'Quentin Tarantino', 1963),
(5, 'Pedro Almodóvar', 1949);


INSERT INTO movie (id, title, production_year, director_id) VALUES
(1, 'Psycho', 1960, 1),
(2, 'Saving Private Ryan', 1998, 2),
(3, 'Schindler''s List', 1993, 2),
(4, 'Midnight in Paris', 2011, 3),
(5, 'Sweet and Lowdown', 1993, 3),
(6, 'Pulp Fiction', 1994, 4),
(7, 'Talk to Her', 2002, 5),
(8, 'The Skin I Live In', 2011, 5);
''')
print(f"Database ready ✅ ({DB_FILE})")


# Working with Multiple Tables

In the previous notebook, you learned how to retrieve data from a single table
and how to filter rows using conditions.

In real-world databases, however, data is rarely stored in just one table.
Instead, information is usually split across **multiple tables** to reduce
redundancy and improve organization.

As a result, it is often necessary to retrieve and combine data from more than
one table at a time.

In this notebook, you will learn how to work with data from multiple tables and
how to join results to answer more complex questions.


## Overview of the Tables

Before working with data from multiple tables, it is important to understand
the structure of the tables used in the examples.

In this notebook, we will work with two tables: `car` and `person`.

The tables have the following columns:

>- person (id, name, age)
>- car (id, brand, model, price, owner_id)

The `car` table stores information about cars, while the `person` table stores
information about their owners. Each car is linked to its owner through the
`owner_id` column in the `car` table, which references the `id` column in the
`person` table.

This relationship allows us to associate each car with the corresponding person.

The relationship between these tables will be used to combine data in the following sections.

## Using Multiple Tables in the FROM Clause

As seen earlier, tables in a relational database are often connected through
shared identifiers.

For example, in our dataset, the `car` table contains an `owner_id` column.
This column references the `id` column in the `person` table, allowing us to
associate each car with its owner.

There are several ways to retrieve data from multiple tables at the same time.
We will begin with the simplest approach.

```sql
SELECT *
FROM person, car;
```

This query lists two tables in the `FROM` clause, separated by a comma.
At this point, the database combines every row from `person` with every row
from `car`.

The result may not be what you expect, so let’s examine it carefully.

In [3]:
# @title Practice 1
validator = make_df_validator_nospoilers(
    expected_hash="f4d077a0a7e220a2d420e8b6d3dca035f25798f4fd1535635ad9b078b95529b0",
    required_cols=['id','title','production_year',
   'director_id',
   'id',
   'name',
   'birth_year'],
    expected_rows=40,
    sort_rows=True,
    sort_cols=True
)
make_sql_runner(
    conn,
    runner_id="ex1",
    description_md="""
### Practice 1
Get all the data from two tables: movie and director.

If there are 8 movies and 5 directors, **how many rows will we get in our result?** Think about it before you click **Run** and check the result.
""",
    validator=validator,
    schema_tables=["movie","director"]
    )



## Joining Tables Using a Condition

When multiple tables are listed in the `FROM` clause without a condition,
SQL combines every row from one table with every row from the other.
This produces all possible row combinations.

For example, if the `person` table contains 5 rows and the `car` table
contains 8 rows, the result will contain 5 × 8 = 40 rows.

This happens because SQL has no information about how the rows in the two
tables are related.

To restrict the result to meaningful combinations, we must specify a
condition that defines how the tables are connected:

```sql
SELECT *
FROM person, car
WHERE person.id = car.owner_id;
````

This condition tells SQL to return only those row combinations where the
`id` value from the `person` table matches the `owner_id` value from the
`car` table. As a result, each car is associated with its correct owner.

When working with multiple tables, column names should be qualified with
the table name to avoid ambiguity. The table name and column name are
separated by a dot (`.`).

For example:

> * **car**.owner_id
>
> * **person**.id



In [4]:
# @title Practice 2
validator = make_df_validator_nospoilers(
    expected_hash="2ff11bbe86f2eb533db433b87be7c0c74d89b5ac3488330c04d8d47a11d75456",
    required_cols=['id',
   'title',
   'production_year',
   'director_id',
   'id',
   'name',
   'birth_year'],
    expected_rows=8,
    sort_rows=True,
    sort_cols=True
)


make_sql_runner(
    conn,
    runner_id="ex2",
    description_md="""
### Practice 2
Select all columns from tables `movie` and `director` in such a way that a movie is shown together with its director.

For the sake of this exercise, join `movie` and `director` tables in this exact order (`movie, director`). Otherwise the validator would not work
""",
    validator=validator,
    schema_tables=["movie","director"]
    )



## The JOIN Keyword

Joining tables is a common operation in SQL.
Because of this, SQL provides a dedicated keyword for it: `JOIN`.

Using `JOIN` allows us to express relationships between tables more clearly
than listing multiple tables in the `FROM` clause with a condition.

> 💡 **Did you know?**  
> SQL supports several types of joins.  
> In this notebook, we will focus on the most basic and commonly used one.

## Joining Tables Using JOIN

The following example shows how to join two tables using the `JOIN` keyword:

```sql
SELECT *
FROM person
JOIN car
  ON person.id = car.owner_id;
```  
In this query, the tables person and car are joined using the `JOIN` keyword.

SQL also needs to know **how** the tables are related. This is specified using
the `ON` keyword, followed by a condition.

The condition `person.id = car.owner_id` tells SQL to join only those rows
where the `id` value from the person table matches the `owner_id` value
from the `car` table.


In [5]:
# @title Practice 3
validator = make_df_validator_nospoilers(
    expected_hash="2ff11bbe86f2eb533db433b87be7c0c74d89b5ac3488330c04d8d47a11d75456",
    required_cols=['id',
   'title',
   'production_year',
   'director_id',
   'id',
   'name',
   'birth_year'],
    expected_rows=8,
    sort_rows=True,
    sort_cols=True
)

make_sql_runner(
    conn,
    runner_id="ex3",
    description_md="""
### Practice 3
Use the new construction **JOIN ... ON** to join rows from the tables **movie** and **director** in such a way that a movie is shown together with its director. Display **all** columns from both tables.
""",
    validator=validator,
    schema_tables=["movie","director"]
    )



## Displaying Specific Columns

In many cases, we do not need all columns from the joined tables.
Instead, we can select only the columns that are relevant to the query.

For example, the following query returns the model of each car and the
name of its owner:


```sql
SELECT
  person.name,
  car.model
FROM person
JOIN car
  ON person.id = car.owner_id;
```  
Instead of using the asterisk (`*`), the query explicitly lists the columns
to be displayed.

When working with multiple tables, column names should be prefixed with
the table name and separated by a dot (`.`). This makes it clear which
table each column belongs to.

In [6]:
# @title Practice 4
validator = make_df_validator_nospoilers(
    expected_hash="ab2532437374282a9ac7b29e3005ddce2cbad196fc57d4e6cdd3d400f9cf90b5",
    required_cols=['name', 'title'],
    expected_rows=8,
    sort_rows=True,
    sort_cols=True
)

make_sql_runner(
    conn,
    runner_id="ex4",
    description_md="""
### Practice 4
Select **director name** and **movie title** from tables **movie** and **director** in such a way that a movie is shown together with its director.
""",
    validator=validator,
    schema_tables=["movie","director"]
    )



## Referring to Columns Without Table Names

In the previous examples, column names were prefixed with their table names.
This is good practice, especially when working with multiple tables.

However, prefixing the table name is **only required** when there is a **risk of
ambiguity**. If a column name appears in more than one table, the table name
must be specified. **If the column name is unique across the tables used in
the query, the table name can be omitted**

For example:

```sql
SELECT
  name,
  model
FROM person
JOIN car
  ON person.id = owner_id;
```
  
In this query, the column `name` exists only in the `person` table, and the
column `model` exists only in the `car` table. Because these column names are
unique, the table names can be omitted.

Similarly, the column `owner_id` exists only in the `car` table, so it does
not need to be prefixed with the table name.

On the other hand, both tables contain a column named `id`. For this reason,
the condition must explicitly reference `person.id` to avoid ambiguity.

In [7]:
# @title Practice 5
validator = make_df_validator_nospoilers(
    expected_hash="ab2532437374282a9ac7b29e3005ddce2cbad196fc57d4e6cdd3d400f9cf90b5",
    required_cols=['name', 'title'],
    expected_rows=8,
    sort_rows=True,
    sort_cols=True
)
make_sql_runner(
    conn,
    runner_id="ex5",
    description_md="""
### Practice 5
Select **director name** and **movie title** from the movie and director tables in such a way that a movie is shown together with its director. Don't write table names in the `SELECT` clause.
""",
    validator=validator,
    schema_tables=["movie","director"]
    )



## Renaming Columns with Aliases

So far, column names have appeared in the result exactly as they are defined in the table.  
Sometimes, however, we want the output to be clearer or more descriptive.

For example, the column named `id` is shown simply as `id` in the result.  
We can change how it appears by giving it a new name.

```sql
SELECT
  person.id AS person_id,
  car.id AS car_id
FROM person
JOIN car
  ON person.id = car.owner_id;
```  
After the column name (for example, `person.id`), we use the keyword AS
followed by the new name we want to display (`person_id`).

This new name is called an **alias**.
An alias is temporary and does not change the actual column name in the database.
It only affects how the column appears in the result of this specific query.

Using aliases is especially useful when multiple tables contain columns
with the same name. By default, the result does not indicate which table
a column comes from.

In this example, both tables have a column called `id`, so we rename them
to `person_id` and `car_id`. This makes the result easier to read and
avoids confusion.



In [8]:
# @title Practice 6
validator = make_df_validator_nospoilers(
    expected_hash="d560152db28ea1dfd1f3ef9917f340a5fc68b915a7be61d42dd9517bdb0d6006",
    required_cols=['movie_title', 'name'],
    expected_rows=8,
    sort_rows=True,
    sort_cols=True
)
make_sql_runner(
    conn,
    default_sql="""SELECT
  title,
  name
FROM movie
JOIN director
  ON director_id = director.id;""",
    runner_id="ex6",
    description_md="""
### Practice 6
In this exercise, show the *title* column as *movie_title*. We wrote the query from the previous exercise in the template, so you just have to add a proper alias.
""",
    validator=validator,
    schema_tables=["movie","director"]
    )



## Filtering the joined tables

So far, our queries have returned all matching rows from the joined tables.  
Often, we want to limit the result to rows that meet an additional condition.

Filtering joined results is often necessary, and the same rules apply as when
filtering a single table.

```sql
SELECT
  person.id,
  car.model
FROM person
JOIN car
  ON person.id = car.owner_id
WHERE person.age < 25;
```
The new part here is the `WHERE` clause. Now we only look for such connections of cars and their owners where the owner is below 25.

When working with multiple tables, it is important to include the table name
in the condition, such as person.age. This avoids ambiguity and makes it
clear which column the condition refers to.



In [9]:
# @title Practice 7
validator = make_df_validator_nospoilers(
    expected_hash="0a7e76a8c2875dcdf8ab7e517184e6e0923083ac67004956627f4529ca17cad3",
    required_cols=['id',
   'title',
   'production_year',
   'director_id',
   'id',
   'name',
   'birth_year'],
    expected_rows=3,
    sort_rows=True,
    sort_cols=True
)

make_sql_runner(
    conn,
    runner_id="ex7",
    description_md="""
### Practice 7
Select all columns from tables *movie* and *director* in such a way that a movie is shown together with its director. Select only those movies which were made after 2000. In the joining condition, let the first table be *movie* and the second table be *director*.
""",
    validator=validator,
    schema_tables=["movie","director"]
    )



## Filtering the joined tables continued

Conditions in the `WHERE` clause can also be based on text values.
For example, the following query selects only cars of a specific brand:

```sql
SELECT
  person.id,
  car.model
FROM person
JOIN car
 ON person.id = car.owner_id
WHERE car.brand = 'Fiat';
```
In this query, the result is restricted to rows where the value of the
brand column in the car table is 'Fiat'.



In [10]:
# @title Practice 8
validator = make_df_validator_nospoilers(
    expected_hash="24f9b4cf85fe2822aac458f0e2ad694e0d9913d0f9f24fac0a6082f2153b87e0",
    required_cols=['id',
   'title',
   'production_year',
   'director_id',
   'id',
   'name',
   'birth_year'],
    expected_rows=2,
    sort_rows=True,
    sort_cols=True
)


make_sql_runner(
    conn,
    runner_id="ex8",
    description_md="""
### Practice 8
Select all columns from tables *movie* and *director* in such a way that a movie is shown together with its director. Select only those movies which were directed by Steven Spielberg.
""",
    validator=validator,
    schema_tables=["movie","director"]
    )



## In-class assignment: Put your skills into practice
Let's put into practice everything we've learned so far. Are you ready? This example is going to be slightly more complicated, so make sure you remember everything from this part of the notebook.

In [11]:
# @title In-class exercise 1

make_sql_runner(
    conn,
    runner_id="inclass1",
    description_md="""
### In-class exercise 1
Select the **title** and **production_year** columns from the **movie** table, and the **name** and **birth_year** columns from the **director** table in such a way that a movie is shown together with its director.

Show the column **birth_year** as **born_in**. Select only those movies which were filmed when their director was younger than 40 (i.e. the difference between **production_year** and **birth_year** must be less than 40).
""",
    schema_tables=["movie","director"]
    )



## In-class exercise 2: Further practice
Excellent! We have one more exercise for you before the end of this part. You are going to write your longest SQL query ever and you'll apply knowledge from both of the notebooks you've mastered so far. Are you ready

In [12]:
# @title In-class exercise 2

make_sql_runner(
    conn,
    runner_id="inclass2",
    description_md="""
### In-class exercise 2
Select the **id**, **title**, and **production_year** columns from the **movie** table, and the **name** and **birth_year** columns from the **director** table in such a way that a movie is shown together with its director. Show the column **birth_year** as **born_in** and the column **production_year** as **produced_in**. Select only those movies:

* whose title contains a letter 'a' and which were filmed after 2000,
**or**
* whose director was born between 1945 and 1995.
""",
    schema_tables=["movie","director"]
    )

